Сделано на основе ноутбука от unsloth

https://unsloth.ai/blog/r1-reasoning

Сделано на основе ноутбука от unsloth

https://unsloth.ai/blog/r1-reasoning

Установим все необходимые библиотеки  
unsloth - Для оптимизации тренировки  
vllm - для инференса модели  
tensorboard - для логирования и визуализации  
trl - библиотека для тренировки LLM 

In [ ]:
!pip install unsloth vllm tensorboard
!pip install --upgrade pillow

In [ ]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)
# Импортируем наши генераторы конкретных типов задач:
from re_rl.tasks.generators import (
    generate_random_linear_task,
    generate_random_quadratic_task,
    generate_random_futoshiki_task,
    generate_random_knights_knaves_task,
    generate_random_contradiction_task,
    generate_random_urn_probability_task,
    # ... при желании остальные ...
)
# Импортируем глобальную функцию compute_reward_for_task (или аналог),
# которая умеет проверять ответ для каждого task_type:
from re_rl.rewards import (
    reward_format_check,
    reward_cot_quality,
    reward_correctness,
)
# Импортируем prompts, чтобы подтянуть инструкции
from re_rl.tasks.prompts import PROMPT_TEMPLATES


Мы хотим попробовать потренировать GRPO дома на 3090/4090 с 24ГБ видеопамяти.
Будем тренировать не всю модель, а LoRA адаптер. В таком режиме веса модели замораживаются, а тренируются дополнительные матрицы, которые затем будут добавлены в целевые веса модели.

С последними обновлениями unsloth для GRPO стало возможным использовать модели прямо с очень большим контекстом. В 3090 влезала 3B модель с 15000 контекстом.

Варьируйте параметры max_seq_len, gpu_memory_utilization если параметры установленные по-умолчанию в память не влезают.
А вообще - варьируйте все и ресечьте)

1.5B моделька с общим контекстом 456 будет трениться на всем сете GSM8K-ru примерно ~ часов.
+ если включать промежуточный евал на тестсете один прогон занимает минут 40-50.

In [ ]:
from unsloth import is_bfloat16_supported
import torch

max_seq_length = 2000 # параметр задает длину контекста модели. Чем больше тем больше памяти будет требоваться и медленне тренироваться
lora_rank = 64 # LoRA ранг 64 - довольно большой, у нас получится ~120 миллионов тренируемых параметров.

# model_name = "Qwen/Qwen2.5-7B-Instruct" # Большая 7B модель
# model_name = "Qwen/Qwen2.5-3B-Instruct" # 3B модель
model_name = "Qwen/Qwen2.5-1.5B-Instruct" # 1.5B модель
# model_name = "Qwen/Qwen2.5-0.5B-Instruct" # 0.5B модель, самая слабая, но быстрее всего учится
# один из важнейших параметров далее - gpu_memory_utilization.
# расчеты из того что у нас доступно 24ГБ видеопамяти. Если меньше или больше - варьируйте значение.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    load_in_4bit = True, # Загружаем модель в 4-бит режиме
    fast_inference = True,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # сколько памяти будет занимать модель на видеокарте, можно варьировать
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # список модулей к которым применяется LoRA
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

In [4]:
###################################################
# Шаг 3: Генерация задач
###################################################

# Допустим, мы хотим потренироваться на ЛИНЕЙНЫХ уравнениях (как пример).
# Напишем функцию, которая сделает N тренировочных и M валидационных:
import random
from typing import List, Dict

# 1) Универсальная функция: build_prompt_for_task
def build_prompt_for_task(task_info: Dict) -> List[Dict]:
    """
    Принимает словарь task_info, в котором есть, как минимум:
      {
        "task_type": str,
        "language": str,   # 'ru' или 'en'
        "problem": str,
        "final_answer": str,
        ... (остальные поля при желании)
      }
    1) Из prompts берем инструкции: PROMPT_TEMPLATES[task_type]["instructions"][language]
    2) Формируем system-msg = инструкции
    3) Формируем user-msg = problem (текст задачи) + metadata
    Возвращаем список сообщений [system_msg, user_msg].
    """
    # Шаг 1: Берём task_type, language
    task_type = task_info["task_type"]
    language = task_info["language"]

    # Шаг 2: Достаем instructions
    instructions = ""
    if task_type in PROMPT_TEMPLATES:
        instructions_dict = PROMPT_TEMPLATES[task_type].get("instructions", {})
        instructions = instructions_dict.get(language, "")

    # Вы можете дополнительно объединить base_system_text + instructions.
    # Пока вставляем только instructions.
    system_msg = {
        "role": "system",
        "content": instructions.strip()
    }

    # Шаг 3: user msg (условие задачи)
    user_msg = {
        "role": "user",
        "content": task_info["problem"],
        "metadata": {
            "task_type": task_type,
            "problem": task_info["problem"],
            "ref_final_answer": task_info["final_answer"]
        }
    }
    return [system_msg, user_msg]


# Аналогично можете сделать make_futoshiki_dataset, make_knights_knaves_dataset, ...
# Или же все объединить в одну функцию с параметром.



SYSTEM_PROMPT = """Отвечай строго в формате:
<reasoning>
(Шаги решения)
</reasoning>
<answer>
(Короткий итоговый ответ)
</answer>
"""

###################################################
# Шаг 4: Преобразование в нужный формат
###################################################

def convert_to_trl_format(dataset_list: List[Dict]):
    """
    Пробегаемся по всем элементам dataset_list,
    вызываем build_prompt_for_task(d), 
    и формируем структуру для GRPOTrainer:
      {
        "prompt": [...],
        "answer": d["final_answer"]
      }
    """
    out = []
    for d in dataset_list:
        # формируем prompt
        prompt_msgs = build_prompt_for_task(d)
        ex = {
            "prompt": prompt_msgs,
            "answer": d["final_answer"]
        }
        out.append(ex)
    return out

###################################################
# список reward-функций, вызывающая
###################################################
reward_funcs_list = [
    reward_format_check,   # проверка, что есть <reasoning>...</reasoning> и <answer>...</answer>
    reward_cot_quality,    # небольшой бонус за содержательный reasoning
    reward_correctness,    # основная награда за правильное решение
]

#############################
# линейные уравнения
#############################
def make_linear_dataset(num_train=200, num_eval=50, language="ru", detail_level=3):
    train_items = []
    for _ in range(num_train):
        t = generate_random_linear_task(language=language, detail_level=detail_level)
        r = t.get_result()
        # Сохраняем нужные поля
        train_items.append({
            "task_type": t.get_task_type(),       # "linear"
            "problem": r["problem"],             # описание задачи
            "prompt": r["prompt"],               # "Задача: ..."
            "solution_steps": r["solution_steps"],
            "final_answer": r["final_answer"],
        })
    eval_items = []
    for _ in range(num_eval):
        t = generate_random_linear_task(language=language, detail_level=detail_level)
        r = t.get_result()
        eval_items.append({
            "task_type": t.get_task_type(),
            "problem": r["problem"],
            "prompt": r["prompt"],
            "solution_steps": r["solution_steps"],
            "final_answer": r["final_answer"],
        })
    return train_items, eval_items

#############################
# квадратичные уравнения
#############################
def make_quadratic_dataset(num_train=200, num_eval=50, language="ru", detail_level=3):
    tr = []
    for _ in range(num_train):
        t = generate_random_quadratic_task(language=language, detail_level=detail_level)
        r = t.get_result()
        tr.append({
            "task_type": t.get_task_type(),  # "quadratic"
            "problem": r["problem"],
            "prompt": r["prompt"],
            "solution_steps": r["solution_steps"],
            "final_answer": r["final_answer"],
        })
    ev = []
    for _ in range(num_eval):
        t = generate_random_quadratic_task(language=language, detail_level=detail_level)
        r = t.get_result()
        ev.append({
            "task_type": t.get_task_type(),
            "problem": r["problem"],
            "prompt": r["prompt"],
            "solution_steps": r["solution_steps"],
            "final_answer": r["final_answer"],
        })
    return tr, ev

#############################
# FutoshikiTask
#############################

def make_futoshiki_dataset(num_train=100, num_eval=20, language="ru", detail_level=5):
    train_list = []
    for _ in range(num_train):
        t = generate_random_futoshiki_task(language=language, detail_level=detail_level)
        r = t.get_result()
        train_list.append({
            "task_type": t.get_task_type(),  # "futoshiki"
            "problem": r["problem"],
            "prompt": r["prompt"],
            "solution_steps": r["solution_steps"],
            "final_answer": r["final_answer"],
        })
    eval_list = []
    for _ in range(num_eval):
        t = generate_random_futoshiki_task(language=language, detail_level=detail_level)
        r = t.get_result()
        eval_list.append({
            "task_type": t.get_task_type(),
            "problem": r["problem"],
            "prompt": r["prompt"],
            "solution_steps": r["solution_steps"],
            "final_answer": r["final_answer"],
        })
    return train_list, eval_list

#############################
# KnightsKnavesTask
#############################
def make_knights_knaves_dataset(num_train=100, num_eval=20, language="en", detail_level=4):
    tr = []
    for _ in range(num_train):
        t = generate_random_knights_knaves_task(language=language, detail_level=detail_level)
        r = t.get_result()
        tr.append({
            "task_type": t.get_task_type(),
            "language": language,   # <--- добавили
            "problem": r["problem"],
            "prompt": r["prompt"],
            "final_answer": r["final_answer"],
        })
    ev = []
    for _ in range(num_eval):
        t = generate_random_knights_knaves_task(language=language, detail_level=detail_level)
        r = t.get_result()
        ev.append({
            "task_type": t.get_task_type(),
            "language": language,   # <--- добавили
            "problem": r["problem"],
            "prompt": r["prompt"],
            "final_answer": r["final_answer"],
        })
    return tr, ev

#############################
# ContradictionTask (пример)
#############################
def make_contradiction_dataset(num_train=100, num_eval=20, language="ru"):
    tr = []
    for _ in range(num_train):
        t = generate_random_contradiction_task(language=language, num_statements=10)
        r = t.get_result()
        tr.append({
            "task_type": t.get_task_type(),  # "contradiction"
            "problem": r["problem"],
            "prompt": r["prompt"],
            "solution_steps": r["solution_steps"],
            "final_answer": r["final_answer"],
        })
    ev = []
    for _ in range(num_eval):
        t = generate_random_contradiction_task(language=language, num_statements=10)
        r = t.get_result()
        ev.append({
            "task_type": t.get_task_type(),
            "problem": r["problem"],
            "prompt": r["prompt"],
            "solution_steps": r["solution_steps"],
            "final_answer": r["final_answer"],
        })
    return tr, ev
#############################
# Шаг 8: UrnProbabilityTask (пример)
#############################
def make_urnprob_dataset(num_train=80, num_eval=20, language="ru"):
    tr = []
    for _ in range(num_train):
        t = generate_random_urn_probability_task(language=language)
        r = t.get_result()
        tr.append({
            "task_type": t.get_task_type(),  # "urn_probability"
            "problem": r["problem"],
            "prompt": r["prompt"],
            "solution_steps": r["solution_steps"],
            "final_answer": r["final_answer"],
        })
    ev = []
    for _ in range(num_eval):
        t = generate_random_urn_probability_task(language=language)
        r = t.get_result()
        ev.append({
            "task_type": t.get_task_type(),
            "problem": r["problem"],
            "prompt": r["prompt"],
            "solution_steps": r["solution_steps"],
            "final_answer": r["final_answer"],
        })
    return tr, ev


###################################################
# Шаг 6: Собственно обучение GRPO на выбранных
###################################################
# 1) генерируем датасет
# train_set, eval_set = make_linear_dataset(num_train=1000, num_eval=50, language="ru", detail_level=3)

# train_set, eval_set = make_quadratic_dataset(num_train=1000, num_eval=50)

# train_set, eval_set = make_futoshiki_dataset(num_train=500, num_eval=20)

train_set, eval_set = make_knights_knaves_dataset(num_train=500, num_eval=20, language="ru", detail_level=4)

# train_set, eval_set = make_contradiction_dataset(num_train=80, num_eval=20, language="ru")

# train_set, eval_set = make_urnprob_dataset(num_train=80, num_eval=20, language="ru")

# 2) конвертируем
train_converted_set = convert_to_trl_format(train_set)
eval_converted_set  = convert_to_trl_format(eval_set)


In [ ]:
print(len(train_converted_set))

In [ ]:
from trl import GRPOConfig, GRPOTrainer, LogCompletionsCallback
# 3) Config
train_args_lin = GRPOConfig(
    use_vllm = True,
    vllm_gpu_memory_utilization = 0.3,
    learning_rate = 2e-5,
    num_train_epochs = 1,
    logging_steps = 1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    save_steps = 50,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4,
    num_generations = 8,
    max_prompt_length = 1000,
    max_completion_length = 1000,
    report_to = "tensorboard", # пишем логи в тензорборд
    output_dir = "outputs_artificial_task",
)

trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = reward_funcs_list,  # список из одной функции
    args = train_args_lin,
    train_dataset = train_converted_set,
    eval_dataset  = eval_converted_set,
)

completions_callback = LogCompletionsCallback(trainer=trainer)
trainer.add_callback(completions_callback)
trainer.train(resume_from_checkpoint=False)

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Сколько раз р встречается в слове стравберри?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

In [ ]:
model.save_lora("grpo_saved_lora")

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Сколько раз р встречается в слове стравберри?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output